# Building Response Synthesis from Scratch

In this tutorial, we show you how to build the "LLM synthesis" component of a RAG pipeline from scratch. Given a set of retrieved Nodes, we'll show you how to synthesize a response even if the retrieved context overflows the context window.

We'll walk through some synthesis strategies:
- Create and Refine
- Tree Summarization

We're essentially unpacking our "Response Synthesis" module and exposing that for the user.

We use OpenAI as a default LLM but you're free to plug in any LLM you wish.

## Setup

We build an empty Pinecone Index, and define the necessary LlamaIndex wrappers/abstractions so that we can load/index data and get back a vector retriever.

#### Load Data

In [ ]:
!mkdir data
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

In [1]:
from pathlib import Path
from llama_hub.file.pymu_pdf.base import PyMuPDFReader

In [2]:
loader = PyMuPDFReader()
documents = loader.load(file_path="./data/llama2.pdf")

#### Build Pinecone Index, Get Retriever

We use our high-level LlamaIndex abstractions to 1) ingest data into Pinecone, and then 2) get a vector retriever.

Note that we set chunk sizes to 1024.

In [3]:
import pinecone
import os

api_key = os.environ["PINECONE_API_KEY"]
pinecone.init(api_key=api_key, environment="us-west1-gcp")

/Users/jerryliu/Programming/gpt_index/.venv/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
# dimensions are for text-embedding-ada-002
pinecone.create_index("quickstart", dimension=1536, metric="euclidean", pod_type="p1")

In [5]:
pinecone_index = pinecone.Index("quickstart")

In [6]:
# [Optional] drop contents in index
pinecone_index.delete(deleteAll=True)

{}

In [7]:
from llama_index.vector_stores import PineconeVectorStore
from llama_index import VectorStoreIndex, ServiceContext
from llama_index.storage import StorageContext

In [8]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
# NOTE: set chunk size of 1024
service_context = ServiceContext.from_defaults(chunk_size=1024)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, service_context=service_context, storage_context=storage_context
)

Upserted vectors:   0%|          | 0/112 [00:00<?, ?it/s]

In [9]:
retriever = index.as_retriever()

#### Given an example question, get a retrieved set of nodes.

We use the retriever to get a set of relevant nodes given a user query. These nodes will then be passed to the response synthesis modules below.

In [10]:
query_str = "Can you tell me about results from RLHF using both model-based and human-based evaluation?"

In [11]:
retrieved_nodes = retriever.retrieve(query_str)

## Building Response Synthesis with LLMs

In this section we'll show how to use LLMs + Prompts to build a response synthesis module.

We'll start from simple strategies (simply stuffing context into a prompt), to more advanced strategies that can handle context overflows.

### 1. Try a Simple Prompt

We first try to synthesize the response using a single input prompt + LLM call.

In [22]:
from llama_index.llms import OpenAI
from llama_index.prompts import PromptTemplate

llm = OpenAI(model="text-davinci-003")

In [13]:
qa_prompt = PromptTemplate("""\
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: \
""")

Given an example question, retrieve the set of relevant nodes and try to put it all in the prompt, separated by newlines.

In [14]:
query_str = "Can you tell me about results from RLHF using both model-based and human-based evaluation?"

In [15]:
retrieved_nodes = retriever.retrieve(query_str)

In [16]:
def generate_response(retrieved_nodes, query_str, qa_prompt, llm):
    context_str = "\n\n".join([r.get_content() for r in retrieved_nodes])
    fmt_qa_prompt = qa_prompt.format(context_str=context_str, query_str=query_str)
    response = llm.complete(fmt_qa_prompt)
    return str(response), fmt_qa_prompt

In [17]:
response, fmt_qa_prompt = generate_response(retrieved_nodes, query_str, qa_prompt, llm)

In [18]:
print(f'*****Response******:\n{response}\n\n')

*****Response******:

RLHF used both model-based and human-based evaluation to select the best-performing models among several ablations. Model-based evaluation was used to measure the robustness of the reward model by collecting a test set of prompts for both helpfulness and safety, and asking three annotators to judge the quality of the answers based on a 7-point Likert scale. Human evaluation was used to validate major model versions. Additionally, a more general reward was trained to ensure the measure wouldn't diverge from the human preferences. Results showed that the reward models were well calibrated with the human preference annotations.




In [19]:
print(f'*****Formatted Prompt*****:\n{fmt_qa_prompt}\n\n')

*****Formatted Prompt*****:
Context information is below.
---------------------
3.4
RLHF Results
3.4.1
Model-Based Evaluation
Evaluating LLMs is a challenging open-research problem. Human evaluation, while a gold standard, can
be complicated by various HCI considerations (Clark et al., 2021; Gehrmann et al., 2023), and is not always
scalable. Thus, to select the best-performing models among several ablations at each iteration from RLHF-V1
to V5, we first observed the improvement of the rewards from the latest reward models, to save costs and
increase iteration speed. We later validated major model versions with human evaluations.
How Far Can Model-Based Evaluation Go?
To measure the robustness of our reward model, we collected
a test set of prompts for both helpfulness and safety, and asked three annotators to judge the quality of the
answers based on a 7-point Likert scale (the higher the better). We observe that our reward models overall
are well calibrated with our human preference 

**Problem**: What if we set the top-k retriever to a higher value? The context would overflow!

In [33]:
retriever = index.as_retriever(similarity_top_k=6)
retrieved_nodes = retriever.retrieve(query_str)

In [34]:
response, fmt_qa_prompt = generate_response(retrieved_nodes, query_str, qa_prompt, llm)
print(f'Response (k=5): {response}')

ValueError: The prompt is too long for the model. Please use a prompt that is less than 4097 tokens.

### 2. Try a "Create and Refine" strategy

To deal with context overflows, we can try a strategy where we synthesize a response sequentially through all nodes. Start with the first node and generate an initial response. Then for subsequent nodes, refine the answer using additional context.

This requires us to define a "refine" prompt as well.

In [35]:
refine_prompt = PromptTemplate("""\
The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer \
(only if needed) with some more context below.
------------
{context_str}
------------
Given the new context, refine the original answer to better answer the query. \
If the context isn't useful, return the original answer.
Refined Answer: \
""")

In [38]:
from llama_index.response.notebook_utils import display_source_node

def generate_response_cr(retrieved_nodes, query_str, qa_prompt, refine_prompt, llm):
    """Generate a response using create and refine strategy.

    The first node uses the 'QA' prompt.
    All subsequent nodes use the 'refine' prompt.
    
    """
    cur_response = None
    fmt_prompts = []
    for idx, node in enumerate(retrieved_nodes):
        print(f"[Node {idx}]")
        display_source_node(node, source_length=2000)
        context_str = node.get_content()
        if idx == 0:
            fmt_prompt = qa_prompt.format(
                context_str=context_str,
                query_str=query_str
            )
        else:
            fmt_prompt = refine_prompt.format(
                context_str=context_str,
                query_str=query_str,
                existing_answer=str(cur_response)
            )

        cur_response = llm.complete(fmt_prompt)
        fmt_prompts.append(fmt_prompt)

    return str(cur_response), fmt_prompts

In [39]:
response, fmt_prompts = generate_response_cr(retrieved_nodes, query_str, qa_prompt, refine_prompt, llm)

[Node 0]


**Node ID:** 966ce8f2-88aa-4ab2-ac90-869c35e050b8<br>**Similarity:** 0.211344719<br>**Text:** 3.4
RLHF Results
3.4.1
Model-Based Evaluation
Evaluating LLMs is a challenging open-research problem. Human evaluation, while a gold standard, can
be complicated by various HCI considerations (Clark et al., 2021; Gehrmann et al., 2023), and is not always
scalable. Thus, to select the best-performing models among several ablations at each iteration from RLHF-V1
to V5, we first observed the improvement of the rewards from the latest reward models, to save costs and
increase iteration speed. We later validated major model versions with human evaluations.
How Far Can Model-Based Evaluation Go?
To measure the robustness of our reward model, we collected
a test set of prompts for both helpfulness and safety, and asked three annotators to judge the quality of the
answers based on a 7-point Likert scale (the higher the better). We observe that our reward models overall
are well calibrated with our human preference annotations, as illustrated in Figure 29 in the appendix. This
confirms the relevance of using our reward as a point-wise metric, despite being trained with a Pairwise
Ranking Loss.
Still, as Goodhart’s Law states, when a measure becomes a target, it ceases to be a good measure. To ensure
our measure won’t diverge from the human preferences, we additionally used a more general reward, trained
17<br>

[Node 1]


**Node ID:** caa3f54a-2659-42ee-b69a-222db569a51f<br>**Similarity:** 0.286175132<br>**Text:** 5
Discussion
Here, we discuss the interesting properties we have observed with RLHF (Section 5.1). We then discuss the
limitations of Llama 2-Chat (Section 5.2). Lastly, we present our strategy for responsibly releasing these
models (Section 5.3).
5.1
Learnings and Observations
Our tuning process revealed several interesting results, such as Llama 2-Chat’s abilities to temporally
organize its knowledge, or to call APIs for external tools.
SFT (Mix)
SFT (Annotation)
RLHF (V1)
0.0
0.2
0.4
0.6
0.8
1.0
Reward Model Score
RLHF (V2)
Figure 20: Distribution shift for progressive versions of Llama 2-Chat, from SFT models towards RLHF.
Beyond Human Supervision.
At the outset of the project, many among us expressed a preference for
supervised annotation, attracted by its denser signal. Meanwhile reinforcement learning, known for its insta-
bility, seemed a somewhat shadowy field for those in the NLP research community. However, reinforcement
learning proved highly effective, particularly given its cost and time effectiveness. Our findings underscore
that the crucial determinant of RLHF’s success lies in the synergy it fosters between humans and LLMs
throughout the annotation process.
Even with proficient annotators, each individual writes with significant variation. A model fine-tuned on
SFT annotation learns this diversity, including, unfortunately, the tail-end of poorly executed annotation. Fur-
thermore, the model’s performance is capped by the writing abilities of the most skilled annotators. Human
annotators are arguably less subject to discrepancy when comparing two outputs’ preference annotation
for RLHF. Consequently, the reward mechanism swiftly learns to assign low scores to undesirable tail-end
distribution and aligns towards the human preference. This phenomena is illustrated in Figure 20, where we
can see that the worst answers are progressively removed, shifting the distribution to the right.
In addition, during annotation, the model has the potential to ven...<br>

[Node 2]


...

**Node ID:** b0fdf736-4959-4a88-9e3b-4ca5bed4c95a<br>**Similarity:** 0.330254316<br>**Text:** sampled human preferences, whereby human annotators select which of two model outputs they prefer.
This human feedback is subsequently used to train a reward model, which learns patterns in the preferences
of the human annotators and can then automate preference decisions.
3.2.1
Human Preference Data Collection
Next, we collect human preference data for reward modeling. We chose a binary comparison protocol over
other schemes, mainly because it enables us to maximize the diversity of collected prompts. Still, other
strategies are worth considering, which we leave for future work.
Our annotation procedure proceeds as follows. We ask annotators to first write a prompt, then choose
between two sampled model responses, based on provided criteria. In order to maximize the diversity, the
two responses to a given prompt are sampled from two different model variants, and varying the temperature
hyper-parameter. In addition to giving participants a forced choice, we also ask annotators to label the degree
to which they prefer their chosen response over the alternative: either their choice is significantly better, better,
slightly better, or negligibly better/ unsure.
For our collection of preference annotations, we focus on helpfulness and safety. Helpfulness refers to how
well Llama 2-Chat responses fulfill users’ requests and provide requested information; safety refers to
whether Llama 2-Chat’s responses are unsafe, e.g., “giving detailed instructions on making a bomb” could
be considered helpful but is unsafe according to our safety guidelines. Separating the two allows us to
apply specific guidelines to each and better guide annotators; for example, our safety annotations provide
instructions to focus on adversarial prompts, among other guidance.
Apart from differences in annotation guidelines, we additionally collect a safety label during the safety stage.
This additional information bins model responses into one of three categories: 1) the preferred response
is saf...<br>

[Node 4]


**Node ID:** b30af56e-775a-4867-8373-dc6528a64c73<br>**Similarity:** 0.333418608<br>**Text:** 1
2
3
4
5
6
7
8
9
10
11
12
13
14
Meta Helpfulness Data Batch Stage
0.52
0.54
0.56
0.58
0.60
0.62
0.64
Accuracy On All Examples
7b
13b
70b
GPT4
OpenAssistant
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Meta Helpfulness Data Batch Stage
0.50
0.55
0.60
0.65
0.70
0.75
0.80
Accuracy On Examples With Label "Significantly Better"
7b
13b
70b
GPT4
OpenAssistant
Figure 6: Scaling trends for the reward model. More data and a larger-size model generally improve
accuracy, and it appears that our models have not yet saturated from learning on the training data.
The fact that helpfulness and safety performed the best on their own domain is potentially due to the tension
between the two objectives (i.e., being as helpful as possible versus refusing unsafe prompts when necessary),
which may confuse the reward model during training. In order for a single model to perform well on both
dimensions, it needs to not only learn to select the better response given a prompt but also to distinguish
adversarial prompts from safe ones. As a result, optimizing two separate models eases the reward modeling
task. More detailed analysis on this tension between safety and helpfulness can be found in Appendix A.4.1.
When we group the scores by preference rating in Table 8, we can see that the accuracy is superior for the
“significantly better” test set and degrades gradually as comparison pairs become more similar (e.g., “slightly
better”). It is expected that learning to model human preferences becomes challenging when deciding
between two similar model responses, due to annotator subjectivity and their reliance on nuanced details
that may differentiate responses. We emphasize that the accuracy on more distinct responses matters the
most to improve Llama 2-Chat performance. The human preference annotation agreement rate is also higher
on more distinct responses than similar pairs.
Scaling Trends.
We study the scaling trends in terms of data and model size for the reward model, fine-
tuning different model s...<br>

[Node 5]


**Node ID:** d48ec89a-3e3d-4887-ad50-1db6b745e544<br>**Similarity:** 0.338611722<br>**Text:** Figure 12: Human evaluation results for Llama 2-Chat models compared to open- and closed-source models
across ~4,000 helpfulness prompts with three raters per prompt.
The largest Llama 2-Chat model is competitive with ChatGPT. Llama 2-Chat 70B model has a win rate of
36% and a tie rate of 31.5% relative to ChatGPT. Llama 2-Chat 70B model outperforms PaLM-bison chat
model by a large percentage on our prompt set. More results and analysis is available in Section A.3.7.
Inter-Rater Reliability (IRR).
In our human evaluations, three different annotators provided independent
assessments for each model generation comparison. High IRR scores (closer to 1.0) are typically seen as
better from a data quality perspective, however, context is important. Highly subjective tasks like evaluating
the overall helpfulness of LLM generations will usually have lower IRR scores than more objective labelling
tasks. There are relatively few public benchmarks for these contexts, so we feel sharing our analysis here will
benefit the research community.
We used Gwet’s AC1/2 statistic (Gwet, 2008, 2014) to measure inter-rater reliability (IRR), as we found it to
be the most stable metric across different measurement scenarios. On the 7-point Likert scale helpfulness
task that is used in our analysis, Gwet’s AC2 score varies between 0.37 and 0.55 depending on the specific
model comparison. We see scores on the lower end of that range for ratings from model comparisons with
similar win rates to each other (like the Llama 2-Chat-70B-chat vs. ChatGPT comparison). We see scores on
the higher end of that range for ratings from model comparisons with a more clear winner (like the Llama
2-Chat-34b-chat vs. Falcon-40b-instruct).
Limitations of human evaluations.
While our results indicate that Llama 2-Chat is on par with ChatGPT
on human evaluations, it is important to note that human evaluations have several limitations.
• By academic and research standards, we have a large prompt set of 4k prompt...<br>

In [40]:
print(str(response))


RLHF results were evaluated using both model-based and human-based evaluation. Model-based evaluation was used to select the best-performing models among several ablations at each iteration from RLHF-V1 to V5. Human evaluation was used to measure the robustness of the reward model, and was done by collecting a test set of prompts for both helpfulness and safety, and asking three annotators to judge the quality of the answers based on a 7-point Likert scale. Additionally, a more general reward was trained to ensure the measure wouldn't diverge from the human preferences. The tuning process revealed several interesting results, such as Llama 2-Chat’s abilities to temporally organize its knowledge, or to call APIs for external tools. 

To collect human preference data for reward modeling, we used a binary comparison protocol, whereby human annotators select which of two model outputs they prefer. This human feedback is subsequently used to train a reward model, which learns patterns in t

In [41]:
# view a sample qa prompt
print(fmt_prompts[0])

Context information is below.
---------------------
3.4
RLHF Results
3.4.1
Model-Based Evaluation
Evaluating LLMs is a challenging open-research problem. Human evaluation, while a gold standard, can
be complicated by various HCI considerations (Clark et al., 2021; Gehrmann et al., 2023), and is not always
scalable. Thus, to select the best-performing models among several ablations at each iteration from RLHF-V1
to V5, we first observed the improvement of the rewards from the latest reward models, to save costs and
increase iteration speed. We later validated major model versions with human evaluations.
How Far Can Model-Based Evaluation Go?
To measure the robustness of our reward model, we collected
a test set of prompts for both helpfulness and safety, and asked three annotators to judge the quality of the
answers based on a 7-point Likert scale (the higher the better). We observe that our reward models overall
are well calibrated with our human preference annotations, as illustrated 

In [42]:
# view a sample refine prompt
print(fmt_prompts[1])

The original query is as follows: Can you tell me about results from RLHF using both model-based and human-based evaluation?
We have provided an existing answer:  RLHF results were evaluated using both model-based and human-based evaluation. Model-based evaluation was used to select the best-performing models among several ablations at each iteration from RLHF-V1 to V5. Human evaluation was used to measure the robustness of the reward model, and was done by collecting a test set of prompts for both helpfulness and safety, and asking three annotators to judge the quality of the answers based on a 7-point Likert scale. Additionally, a more general reward was trained to ensure the measure wouldn't diverge from the human preferences.
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
5
Discussion
Here, we discuss the interesting properties we have observed with RLHF (Section 5.1). We then discuss the
limitations of Llama 2-Chat

**Observation**: This is an initial step, but obviously there are inefficiencies. One is the fact that it's quite slow - we make sequential calls. The second piece is that each LLM call is inefficient - we are only inserting a single node, but not "stuffing" the prompt with as much context as necessary.

### 3. Try a Hierarchical Summarization Strategy

Another approach is to try a hierarchical summarization strategy. We generate an answer for each node independently, and then hierarchically combine the answers. This "combine" step could happen once, or for maximum generality can happen recursively until there is one "root" node. That "root" node is then returned as the answer.

We implement this approach below. We have a fixed number of children of 5, so we hierarchically combine 5 children at a time.

**NOTE**: In LlamaIndex this is referred to as "tree_summarize", in LangChain this is referred to as map-reduce.

In [51]:
def combine_results(
    texts, 
    query_str, 
    qa_prompt, 
    llm, 
    cur_prompt_list,
    num_children=10,
):
    new_texts = []
    for idx in range(0, len(texts), num_children):
        text_batch = texts[idx:idx+num_children]
        context_str = "\n\n".join([t for t in text_batch])
        fmt_qa_prompt = qa_prompt.format(context_str=context_str, query_str=query_str)
        combined_response = llm.complete(fmt_qa_prompt)
        new_texts.append(str(combined_response))
        cur_prompt_list.append(fmt_qa_prompt)
    
    if len(new_texts) == 1:
        return new_texts[0]
    else:
        return combine_results(
            new_texts,
            query_str,
            qa_prompt,
            llm,
            num_children=num_children
        )


def generate_response_hs(retrieved_nodes, query_str, qa_prompt, llm, num_children=10):
    """Generate a response using hierarchical summarization strategy.

    Combine num_children nodes hierarchically until we get one root node.
    
    """
    fmt_prompts = []
    node_responses = []
    for node in retrieved_nodes:
        context_str = node.get_content()
        fmt_qa_prompt = qa_prompt.format(context_str=context_str, query_str=query_str)
        node_response = llm.complete(fmt_qa_prompt)
        node_responses.append(node_response)
        fmt_prompts.append(fmt_qa_prompt)

    response_txt = combine_results(
        [str(r) for r in node_responses], 
        query_str, 
        qa_prompt, 
        llm, 
        fmt_prompts,
        num_children=num_children
    )

    return response_txt, fmt_prompts

In [52]:
response, fmt_prompts = generate_response_hs(
    retrieved_nodes, query_str, qa_prompt, llm
)

In [53]:
print(str(response))


The results from RLHF using both model-based and human-based evaluation showed that Llama 2-Chat models outperformed open-source models by a significant margin on both single turn and multi-turn prompts. For human-based evaluation, we compared Llama 2-Chat models to open-source models and closed-source models on over 4,000 single and multi-turn prompts. The results showed that Llama 2-Chat models outperformed the other models by a significant margin on both single turn and multi-turn prompts. The human preference annotation agreement rate was also higher on more distinct responses than similar pairs. The largest RLHF model was competitive with ChatGPT, with a win rate of 36% and a tie rate of 31.5% relative to ChatGPT. RLHF 70B model also outperformed PaLM-bison chat model by a large percentage on the prompt set.


**Observation**: Note that the answer is much more concise than the create-and-refine approach. This is a well-known phemonenon - the reason is because hierarchical summarization tends to compress information at each stage, whereas create and refine encourages adding on more information with each node.

**Observation**: Similar to the above section, there are inefficiencies. We are still generating an answer for each node independently that we can try to optimize away.

Our `ResponseSynthesizer` module handles this!

#### 4. [Optional] Let's create an async version of hierarchical summarization!

A pro of the hierarchical summarization approach is that the LLM calls can be parallelized, leading to big speedups in response synthesis.

We implement an async version below. We use asyncio.gather to execute coroutines (LLM 

In [71]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

In [74]:
async def acombine_results(
    texts, 
    query_str, 
    qa_prompt, 
    llm, 
    cur_prompt_list,
    num_children=10,
):

    fmt_prompts = []
    for idx in range(0, len(texts), num_children):
        text_batch = texts[idx:idx+num_children]
        context_str = "\n\n".join([t for t in text_batch])
        fmt_qa_prompt = qa_prompt.format(context_str=context_str, query_str=query_str)
        fmt_prompts.append(fmt_qa_prompt)

    tasks = [llm.acomplete(p) for p in fmt_prompts]
    combined_responses = await asyncio.gather(*tasks)
    new_texts = [str(r) for r in combined_responses]
    
    if len(new_texts) == 1:
        return new_texts[0]
    else:
        return await acombine_results(
            new_texts,
            query_str,
            qa_prompt,
            llm,
            num_children=num_children
        )


async def agenerate_response_hs(retrieved_nodes, query_str, qa_prompt, llm, num_children=10):
    """Generate a response using hierarchical summarization strategy.

    Combine num_children nodes hierarchically until we get one root node.
    
    """
    fmt_prompts = []
    node_responses = []
    for node in retrieved_nodes:
        context_str = node.get_content()
        fmt_qa_prompt = qa_prompt.format(context_str=context_str, query_str=query_str)
        fmt_prompts.append(fmt_qa_prompt)

    tasks = [llm.acomplete(p) for p in fmt_prompts]
    node_responses = await asyncio.gather(*tasks)

    response_txt = combine_results(
        [str(r) for r in node_responses], 
        query_str, 
        qa_prompt, 
        llm, 
        fmt_prompts,
        num_children=num_children
    )

    return response_txt, fmt_prompts

In [92]:
response, fmt_prompts = await agenerate_response_hs(
    retrieved_nodes, query_str, qa_prompt, llm
)

In [93]:
print(str(response))

 Results from RLHF using both model-based and human-based evaluation show that larger models generally obtain higher performance for a similar volume of data. Additionally, the accuracy on more distinct responses matters the most to improve Llama 2-Chat performance. The human preference annotation agreement rate is also higher on more distinct responses than similar pairs. Furthermore, two main algorithms were explored for RLHF fine-tuning: Proximal Policy Optimization (PPO) and Rejection Sampling fine-tuning. The largest Llama 2-Chat model was found to be competitive with ChatGPT, with a win rate of 36% and a tie rate of 31.5% relative to ChatGPT. Additionally, Llama 2-Chat 70B model outperformed PaLM-bison chat model by a large percentage on our prompt set. Inter-Rater Reliability (IRR) was measured using Gwet’s AC1/2 statistic, with scores varying between 0.37 and 0.55 depending on the specific model comparison.


## Let's put it all together!

Let's define a simple query engine that can be initialized with a retriever, prompt, llm etc. And have it implemen a simple `query` function.

**NOTE**: We skip subclassing our own `QueryEngine` abstractions. This is a big TODO to make it more easily sub-classable! 

In [86]:
from llama_index.retrievers import BaseRetriever
from llama_index.llms.base import LLM
from dataclasses import dataclass
from typing import Optional, List

@dataclass
class Response:
    response: str
    source_nodes: Optional[List] = None

    def __str__(self):
        return self.response


class MyQueryEngine:
    """My query engine.

    Uses the tree summarize response synthesis module by default.
    
    """
    def __init__(
        self,
        retriever: BaseRetriever,
        qa_prompt: PromptTemplate,
        llm: LLM,
        num_children=10
    ) -> None:
        self._retriever = retriever
        self._qa_prompt = qa_prompt
        self._llm = llm
        self._num_children = num_children 

    def query(self, query_str: str):
        retrieved_nodes = self._retriever.retrieve(query_str)
        response_txt, _ = generate_response_hs(
            retrieved_nodes,
            query_str,
            self._qa_prompt,
            self._llm,
            num_children=self._num_children
        )
        response = Response(response_txt, source_nodes=retrieved_nodes)
        return response


    async def aquery(self, query_str: str):
        retrieved_nodes = await self._retriever.aretrieve(query_str)
        response_txt, _ = await agenerate_response_hs(
            retrieved_nodes,
            query_str,
            self._qa_prompt,
            self._llm,
            num_children=self._num_children
        )
        response = Response(response_txt, source_nodes=retrieved_nodes)
        return response
        

In [87]:
query_engine = MyQueryEngine(
    retriever,
    qa_prompt,
    llm,
    num_children=10
)

In [88]:
response = query_engine.query(query_str)

In [89]:
print(str(response))


The results from RLHF using both model-based and human-based evaluation showed that larger models generally obtained higher performance for a similar volume of data. The accuracy on more distinct responses was higher than on similar pairs, indicating that learning to model human preferences becomes challenging when deciding between two similar model responses. Additionally, the largest Llama 2-Chat model was found to be competitive with ChatGPT, with a win rate of 36% and a tie rate of 31.5% relative to ChatGPT. Llama 2-Chat 70B model was also found to outperform PaLM-bison chat model by a large percentage on the prompt set. Inter-Rater Reliability (IRR) was measured using Gwet’s AC1/2 statistic, with scores varying between 0.37 and 0.55 depending on the specific model comparison.


In [90]:
response = await query_engine.aquery(query_str)

In [91]:
print(str(response))


The results from RLHF using both model-based and human-based evaluation showed that larger models generally obtained higher performance for a similar volume of data. The accuracy on more distinct responses was higher than on similar pairs, indicating that learning to model human preferences becomes challenging when deciding between two similar model responses. Additionally, the largest Llama 2-Chat model was found to be competitive with ChatGPT, with a win rate of 36% and a tie rate of 31.5%. Human evaluations were conducted using a 7-point Likert scale helpfulness task, with Gwet’s AC2 score varying between 0.37 and 0.55 depending on the specific model comparison.
